In [8]:
import os
import numpy as np
import pandas as pd
from datetime import timedelta
from sqlalchemy import create_engine
from dotenv import load_dotenv
from utils import get_db_connection, log

load_dotenv()

def load_data():
    with get_db_connection() as conn:
        esg = pd.read_sql("SELECT * FROM esg_events", conn)
        sat = pd.read_sql("SELECT * FROM satellite_signals", conn)
    return esg, sat

def get_company_sites():
    return pd.DataFrame([
        {"company": "Caterpillar", "site_id": "TX1", "lat": 32.8721, "lon": -96.9391, "region": "Irving, Texas"},
        {"company": "Caterpillar", "site_id": "IL1", "lat": 40.565, "lon": -89.625, "region": "Peoria, Illinois"},
        {"company": "Caterpillar", "site_id": "IL2", "lat": 40.666, "lon": -89.580, "region": "East Peoria, Illinois"},
        {"company": "Caterpillar", "site_id": "IL3", "lat": 40.848, "lon": -89.611, "region": "Mossville, Illinois"},
        {"company": "Caterpillar", "site_id": "GA1", "lat": 33.949, "lon": -83.537, "region": "Athens, Georgia"},
        {"company": "Caterpillar", "site_id": "CN1", "lat": 34.204, "lon": 117.284, "region": "Xuzhou, China"},
        {"company": "Caterpillar", "site_id": "CN2", "lat": 31.491, "lon": 120.311, "region": "Wuxi, China"},
        {"company": "Caterpillar", "site_id": "JP1", "lat": 34.644, "lon": 134.997, "region": "Akashi, Japan"},
        {"company": "Caterpillar", "site_id": "UK1", "lat": 54.756, "lon": -1.336, "region": "Peterlee, United Kingdom"},
        {"company": "Caterpillar", "site_id": "IN1", "lat": 13.082, "lon": 80.271, "region": "Thiruvallur, India"},
        {"company": "Caterpillar", "site_id": "BR1", "lat": -23.550, "lon": -46.633, "region": "São Paulo, Brazil"},
        {"company": "Caterpillar", "site_id": "TX2", "lat": 29.7604, "lon": -95.3698, "region": "Houston, Texas"},
        {"company": "Caterpillar", "site_id": "GA2", "lat": 33.7490, "lon": -84.3880, "region": "Atlanta, Georgia"},
        {"company": "Caterpillar", "site_id": "IN2", "lat": 11.1271, "lon": 78.6569, "region": "Tamil Nadu, India"},
        {"company": "Caterpillar", "site_id": "MX1", "lat": 25.6866, "lon": -100.3161, "region": "Monterrey, Mexico"},
        {"company": "Caterpillar", "site_id": "AU1", "lat": -31.9505, "lon": 115.8605, "region": "Perth, Australia"}
    ])

def build_features(esg, sat, company_sites):
    esg["date"] = pd.to_datetime(esg["date"])
    sat["acq_date"] = pd.to_datetime(sat["acq_date"])
    features = []

    for _, site in company_sites.iterrows():
        esg_site = esg[esg["location"].str.contains(site["region"].split(",")[0], case=False, na=False)]
        sat_site = sat[
            (np.abs(sat["latitude"] - site["lat"]) <= 2.0) &
            (np.abs(sat["longitude"] - site["lon"]) <= 2.0)
        ]

        for day in pd.date_range("2024-01-01", "2025-06-30", freq="W"):
            esg_count = esg_site[(esg_site["date"] >= day - timedelta(days=7)) & (esg_site["date"] <= day)].shape[0]
            avg_frp = sat_site[(sat_site["acq_date"] >= day - timedelta(days=7)) & (sat_site["acq_date"] <= day)]["frp"].mean()

            features.append({
                "company": site["company"],
                "site_id": site["site_id"],
                "date": day,
                "esg_event_count_7d": esg_count,
                "avg_frp_7d": avg_frp if not np.isnan(avg_frp) else 0.0
            })

    return pd.DataFrame(features)

def save_features_to_db(df):
    db_url = os.getenv("DATABASE_URL").replace("postgresql+psycopg2", "postgresql")
    engine = create_engine(db_url)
    df.to_sql("risk_features", engine, index=False, if_exists="replace")
    log("✔ risk_features saved to PostgreSQL.")

if __name__ == "__main__":
    log("🚀 Starting feature engineering...")
    esg, sat = load_data()
    company_sites = get_company_sites()
    df = build_features(esg, sat, company_sites)
    save_features_to_db(df)


[LOG] 🚀 Starting feature engineering...


/var/folders/3y/jcf5sjb1319blyc1b_cfn1xm0000gn/T/ipykernel_25178/2999396115.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  esg = pd.read_sql("SELECT * FROM esg_events", conn)
/var/folders/3y/jcf5sjb1319blyc1b_cfn1xm0000gn/T/ipykernel_25178/2999396115.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sat = pd.read_sql("SELECT * FROM satellite_signals", conn)


[LOG] ✔ risk_features saved to PostgreSQL.
